In [1]:
# Includes libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("data/AppleFinalData.csv")
df.head()

,Date,Open,High,Low,Close,Adj Close,compound,neg,neu,pos
0,2006-12-01,13.1143,13.1900,12.8714,91.32,13.0457,0.7707,0.032,0.905,0.063
1,2006-12-04,13.1257,13.1500,12.9286,91.12,13.0171,0.8720,0.011,0.904,0.085
2,2006-12-05,13.0929,13.1900,12.9814,91.27,13.0386,0.0000,0.000,0.000,0.000
3,2006-12-06,12.9486,13.0557,12.8100,89.83,12.8329,0.6858,0.029,0.878,0.093
4,2006-12-07,12.8614,12.9286,12.4143,87.04,12.4343,-0.6712,0.091,0.869,0.040


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2517 entries, 0 to 2516
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       2517 non-null   object 
 1   Open       2517 non-null   float64
 2   High       2517 non-null   float64
 3   Low        2517 non-null   float64
 4   Close      2517 non-null   float64
 5   Adj Close  2517 non-null   float64
 6   compound   2517 non-null   float64
 7   neg        2517 non-null   float64
 8   neu        2517 non-null   float64
 9   pos        2517 non-null   float64
dtypes: float64(9), object(1)
memory usage: 196.8+ KB
